### Die Klasse Game
Ein Stein kann auf einem der drei Positionen 0, 1 oder 2 liegen.
Der Spielzustand wird durch eine Zahl repr&auml;sentiert, die Position auf welcher der Stein liegt.
- `new_game()` legt den Stein auf Position 0,
- `move(<target>)` verschiebt den Stein auf Position `<target>`.

In [33]:
class Game:

    positions = (0, 1, 2)

    def __init__(self):
        self.position = None
        self.event_handler = print

    def new_game(self):
        self.position = 0
        self.event_handler('new_game', self.position)

    def move(self, target):
        if target in self.positions and self.position != target:
            self.position = target
            self.event_handler('move_stone', target)

    def __repr__(self):
        return str(self.position)

In [34]:
game = Game()
game

None

In [35]:
game.new_game()
game

new_game 0


0

In [36]:
game.move(4)
game

0

### Die Klasse View
Der Spielzustand wird dargestellt, indem
auf der Leinwand der Stein an einer der Positionen
> `[(20, 40), (50, 50), (80, 50)]`

gezeichnet wird.
Diese Positionen sind durch kleine blaue Kreise markiert.

Die Methode `event_handler(event, idx)` aktuallisiert den Spielzustand.
Ist `event` gleich `'new game'` wird der Text 'New Game' angezeigt, 
sonst (falls  `event` gleich `'move'` ist) wird der Stein an der
Position `idx` gezeichnet.  

Ein View-Object wird mit dem Aufruf `View(game)` erstellt, wobei `game` ein Game-Objekt ist.
Die `init`-Methode der View setzt `game.event_handler` gleich
der Methode `event_handler` der View, welche den
Spielzustand auf der Leinwand aktuallisiert.

In [37]:
import widgets_helpers
from IPython.display import display


class View:
    positions = [(20, 40), (50, 50), (80, 50)]
    pt_size = 2
    stone_size = 10

    def __init__(self, game):
        game.event_handler = self.event_handler

        self.mcanvas = widgets_helpers.new_mcanvas(2, 100, 100)
        self.bg, self.fg = self.mcanvas
        self.fg.fill_style = 'blue'
        for x, y in self.positions:
            self.bg.fill_circle(x, y, self.pt_size)

    def place_stone(self, idx):
        self.fg.clear()
        x, y = self.positions[idx]
        self.fg.fill_circle(x, y, self.stone_size)

    def event_handler(self, event, idx):
        self.fg.clear()
        self.place_stone(idx)
        if event == 'new_game':
            self.fg.fill_text('New Game', 20, 20)

    def _ipython_display_(self):
        display(self.mcanvas)

In [38]:
view = View(game)
view

MultiCanvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

In [44]:
# Methode place_stone testen
view.place_stone(1)

In [45]:
# da game.event_handler nun view.event_handler ist,
# wird der Spielzustand auf der Leinwand aktuallisiert
game.new_game()

In [46]:
game.move(2)

### Die Klasse Controller
Der Controller benutzt die Leinwand der View um auf Tastendr&uuml;cke und
Mausklicks zu h&ouml;ren.  
Ein Controller-Objekt wird mit dem Aufruf `Controller(game, view)` erstellt, wobei
`game` und `view` Game- bez View-Objekte sind.


- Wird 'n' gedr&uuml;ckt, wird `game.new_game()` aufgerufen.
- Wird auf die Leinwand geklickt, so wird geschaut, ob blaue Punkte in der N&auml;he liegen, und
  gegebenenfalls auf dem n&auml;chsten Punkt ein Stein platziert. 

In [55]:
from get_closest import get_closest


class Controller:
    err_out = widgets_helpers.new_output()

    def __init__(self, game, view):
        self.game = game

        self.view = view
        self.canvas = view.mcanvas
        self.canvas.on_key_down(self.on_key_down)
        self.canvas.on_mouse_down(self.on_mouse_down)

    @err_out.capture()
    def on_key_down(self, key, *flags):
        if key == 'n':
            self.game.new_game()

    @err_out.capture()
    def on_mouse_down(self, x, y):
        idx = get_closest(self.view.positions, (x, y), err=10)
        if idx is not None and idx != self.game.position:
            self.game.move(idx)

    def _ipython_display_(self):
        display(self.view, self.err_out)

In [56]:
controller = Controller(game, view)
controller

MultiCanvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…